# Supplementary - Dynamics

Contains analyses related to Supplementary Figure S2.

In [ ]:
import sys
sys.path.append("..")

from main import *

from scipy.stats import pearsonr, spearmanr
from tqdm import tqdm
import sys
from scipy.optimize import linear_sum_assignment
import matplotlib.pyplot as plt
from numba import njit
from scipy.stats import zscore

plt.rcParams['font.size'] = 18

if torch.cuda.is_available():  
    device = "cuda:0" 
else:  
    device = "cpu" 
    
print(device)

# Part 1: Inspecting the different dynamics

Here we integrate 5 different dynamical models in the ellipsoid geometry, display some exemplay dynamics, and compute their eigenmode-gradient correlations, just for visualization purposes.

# Dynamics #1: Standard $\tanh$ activation function

In [ ]:
vertices = np.load('../Files/vertices_ellipse.npy').astype('float')
eigenmodes = np.load('../Files/eigenmodes_ellipse.npy')

ellipse = Geometry(vertices, eigenmodes)

In [ ]:
params = {'N_neurons': 1500,
          'h': 0.1,
          'g': 3,
          'tau': 3,
          'dale': False
          }

simulator = Simulator(ellipse,
                      ChaoticRNN,
                      params)

simulator.integrate(1000, output=False)

In [ ]:
%matplotlib inline
simulator.plot(spacing=1.5)

In [ ]:
%matplotlib inline
simulator.imshow(vmin=0, vmax=1)

In [ ]:
save = False
if save:
    np.save('../Results/supp_dynamics_timeseries_dynamics1.npy', simulator.timeseries)

In [ ]:
%matplotlib qt
simulator.animate(alpha=0.75,
                  cmap='hot',
                  vmax=1.5)

simulator.save_animation('Figures/Gifs/chaotic_RNN.gif', fps=50)

In [ ]:
C = simulator.compute_average_correlations(n_iters=100, T=500)

In [ ]:
%matplotlib inline
plt.figure(figsize=(5, 5))
plt.imshow(C, cmap='hot', vmin=0.1, vmax=0.3)

In [ ]:
mode_similarity, mapping = simulator.compute_geometric_mapping(C, N_modes=20)

In [ ]:
%matplotlib inline
fig, ax = plt.subplots(figsize=(5, 5))
cax = plt.imshow(np.abs(mode_similarity), cmap='Reds', vmin=0, vmax=1)
plt.xlabel('Functional modes')
plt.ylabel('Geometric modes')
plt.colorbar(cax, ax=ax, fraction=0.045, pad=0.02)

In [ ]:
save = False
if save:
    np.save('../Results/supp_dynamics_correlations_dynamics1.npy', C)
    np.save('../Results/supp_dynamics_modesimilarity_dynamics1.npy', mode_similarity)

# Model #2: Generalized `Tanh` function

In [ ]:
params = {'N_neurons': 1500,
          'h': 0.1,
          'g': 3,
          'tau': 3,
          'x_s': 0.1,
          'dale': True,
          'g_I': 1
          }

#np.random.seed(400)

simulator = Simulator(ellipse,
                      AdjustedChaoticRNN,
                      params)

#simulator.dynamics.W[simulator.dynamics.W < 0] *= 1.25

simulator.integrate(1000, output=False)

In [ ]:
%matplotlib inline
simulator.plot(spacing=1.5)

In [ ]:
%matplotlib inline
simulator.imshow()

In [ ]:
save = False
if save:
    np.save('../Results/supp_dynamics_timeseries_dynamics2.npy', simulator.timeseries)

In [ ]:
%matplotlib qt
simulator.animate(alpha=0.75,
                  cmap='hot',
                  vmax=2.5)

simulator.save_animation('Figures/Gifs/adjusted_chaotic_RNN.gif', fps=50)

In [ ]:
C = simulator.compute_average_correlations(n_iters=100, T=500)

In [ ]:
%matplotlib inline
plt.figure(figsize=(5, 5))
plt.imshow(C, cmap='hot')

In [ ]:
mode_similarity, mapping = simulator.compute_geometric_mapping(C, N_modes=20)

In [ ]:
%matplotlib inline
fig, ax = plt.subplots(figsize=(5, 5))
cax = plt.imshow(np.abs(mode_similarity), cmap='Reds', vmin=0, vmax=1)
plt.xlabel('Functional modes')
plt.ylabel('Geometric modes')
plt.colorbar(cax, ax=ax, fraction=0.045, pad=0.02)

In [ ]:
save = False
if save:
    np.save('../Results/supp_dynamics_correlations_dynamics2.npy', C)
    np.save('../Results/supp_dynamics_modesimilarity_dynamics2.npy', mode_similarity)

# Dynamics #3: Kuramoto-Sakaguchi model

In [ ]:
params = {'N_neurons': 1500,
          'h': 0.1,
          'coupling': 2000,
          'tau': 100,
          'alpha': 50,
          'std': 0.1
          }

np.random.seed(420)

simulator = Simulator(ellipse,
                      KuramotoSakaguchi,
                      params)

simulator.integrate(1100, output=False)

In [ ]:
%matplotlib inline
simulator.plot(spacing=1)

In [ ]:
%matplotlib inline
simulator.imshow()

In [ ]:
save = False
if save:
    np.save('../Results/supp_dynamics_timeseries_dynamics3.npy', simulator.timeseries)

In [ ]:
%matplotlib qt
simulator.animate(alpha=0.75,
                  cmap='hot',
                  vmax=1.5)

simulator.save_animation('Figures/Gifs/kuramoto_sakaguchi.gif', fps=50)

In [ ]:
C = simulator.compute_average_correlations(n_iters=100, T=500)

In [ ]:
%matplotlib inline
plt.figure(figsize=(5, 5))
plt.imshow(C, cmap='hot')

In [ ]:
mode_similarity, mapping = simulator.compute_geometric_mapping(C, N_modes=20)

In [ ]:
%matplotlib inline
fig, ax = plt.subplots(figsize=(5, 5))
cax = plt.imshow(np.abs(mode_similarity), cmap='Reds')
plt.xlabel('Functional modes')
plt.ylabel('Geometric modes')
plt.colorbar(cax, ax=ax, fraction=0.045, pad=0.02)

In [ ]:
save = False
if save:
    np.save('../Results/supp_dynamics_correlations_dynamics3.npy', C)
    np.save('../Results/supp_dynamics_modesimilarity_dynamics3.npy', mode_similarity)

#### Sweeping `alpha` parameter

Doing a linear sweep of the `alpha` parameter to find a regime where eigenmodes and gradients are correlated to each other.

In [ ]:
alpha_values = np.linspace(0, 200, 40, endpoint=True)

scores = []

for alpha in tqdm(alpha_values):

    params['alpha'] = alpha
    
    simulator = Simulator(ellipse,
                      KuramotoSakaguchi,
                      params)
    
    C = simulator.compute_average_correlations(n_iters=25, T=500, verbose=False)
    mode_similarity = simulator.compute_geometric_mapping(C, N_modes=20)

    scores.append(np.mean(np.abs(np.diag(mode_similarity))))

In [ ]:
alpha_values[np.argmax(scores)]

In [ ]:
plt.plot(alpha_values, scores)

#### Sweeping coupling values

Doing a linear sweep of the coupling parameter to find a regime where eigenmodes and gradients are correlated to each other.

In [ ]:
params['alpha'] = 50
coupling_values = np.linspace(0, 2000, 40, endpoint=True)

scores = []

for coupling in tqdm(coupling_values):

    params['coupling'] = coupling
    
    simulator = Simulator(ellipse,
                      KuramotoSakaguchi,
                      params)
    
    C = simulator.compute_average_correlations(n_iters=5, T=400, verbose=False)
    mode_similarity = simulator.compute_geometric_mapping(C, N_modes=20)

    scores.append(np.mean(np.abs(np.diag(mode_similarity))))

In [ ]:
plt.plot(coupling_values, scores)

# Dynamics #4: Binary neuron cascades

Discrete dynamical system similar to epidemic spreading.
- `P_activation`: Probability of becoming active/infected, proportional to the number of active/infected neighbors.
- `P_deactivation`: Probability of spontaneously becoming inactive/uninfected.

In [ ]:
params = {'N_neurons': 1500,
          'h': 0.1,
          'P_activation': 0.15,
          'P_deactivation': 0.5
          }

np.random.seed(210)

simulator = Simulator(ellipse,
                      BinaryNetwork,
                      params)

simulator.integrate(1000, output=False)

In [ ]:
%matplotlib inline
simulator.plot(spacing=1)

In [ ]:
%matplotlib inline
simulator.imshow()

np.save('Results/supp_dynamics_timeseries_dynamics4.npy', simulator.timeseries)

In [ ]:
%matplotlib qt
simulator.animate(alpha=0.75,
                  cmap='hot',
                  vmax=1.25, fps=30)

simulator.save_animation('Figures/Gifs/binary_propagation.gif', fps=30)

In [ ]:
C = simulator.compute_average_correlations(n_iters=100, T=500)

In [ ]:
%matplotlib inline
plt.figure(figsize=(5, 5))
plt.imshow(C, cmap='hot')

In [ ]:
mode_similarity, mapping = simulator.compute_geometric_mapping(C, N_modes=20)

In [ ]:
%matplotlib inline
fig, ax = plt.subplots(figsize=(5, 5))
cax = plt.imshow(np.abs(mode_similarity), cmap='Reds')
plt.xlabel('Functional modes')
plt.ylabel('Geometric modes')
plt.colorbar(cax, ax=ax, fraction=0.045, pad=0.02)

In [ ]:
np.mean(np.abs(np.diag(mode_similarity)))

In [ ]:
np.save('../Results/supp_dynamics_correlations_dynamics4.npy', C)
np.save('../Results/supp_dynamics_modesimilarity_dynamics4.npy', mode_similarity)

# Dynamics #5: Spiking neural network

In [ ]:
vertices = np.load('../Files/vertices_ellipse.npy').astype('float')
eigenmodes = np.load('../Files/eigenmodes_ellipse.npy')

ellipse = Geometry(vertices, eigenmodes)

In [ ]:
params = {'N_neurons': 1500,
         'h': 0.1,
         'g_L': 0.15,
         'E_L': -75,
         'E_E': -40,
         'E_I': -90,
         'V_thresh':-55,
         'V_reset':-75,
         'tau': 0.1,
         'I_E': 5,
         'I_I': 0,
         'sigma_E': 0,
         'sigma_I': 0,
         'delta_t': 0.025
         }

simulator = Simulator(ellipse,
                      LIFNetwork,
                      params)

simulator.integrate(1100, output=False)

In [ ]:
simulator.timeseries = gcampify(simulator.timeseries, tau=10)

In [ ]:
%matplotlib inline
simulator.plot(spacing=1.5)

In [ ]:
%matplotlib inline
simulator.imshow(vmin=0)

In [ ]:
np.save('../Results/supp_dynamics_timeseries_dynamics5.npy', simulator.timeseries)

In [ ]:
%matplotlib qt
simulator.animate(alpha=0.75,
                  cmap='hot',
                 vmax=0.15)

simulator.save_animation('spiking.gif', fps=50)

In [ ]:
C = simulator.compute_average_correlations(n_iters=100, T=500, calcium=True, tau_calcium=10)

In [ ]:
%matplotlib inline
plt.figure(figsize=(5, 5))
plt.imshow(C, cmap='hot', vmax=0.2, vmin=0.02)

In [ ]:
mode_similarity, mapping = simulator.compute_geometric_mapping(C, N_modes=20)

In [ ]:
%matplotlib inline
fig, ax = plt.subplots(figsize=(5, 5))
cax = plt.imshow(np.abs(mode_similarity), cmap='Reds')
plt.xlabel('Functional modes')
plt.ylabel('Geometric modes')
plt.colorbar(cax, ax=ax, fraction=0.045, pad=0.02)

In [ ]:
np.save('../Results/supp_dynamics_correlations_dynamics5.npy', C)
np.save('../Results/supp_dynamics_modesimilarity_dynamics5.npy', mode_similarity)

# Part 2: Computing over many simulations

Now, we run many simulations with each dynamical model in the optimal parameter regime to compute an average eigenmode-gradient mapping in the ellipsoid geometry.

Loading geometry

In [ ]:
vertices = np.load('../Files/vertices_ellipse.npy').astype('float')
eigenmodes = np.load('../Files/eigenmodes_ellipse.npy')

ellipse = Geometry(vertices, eigenmodes)

Setting up dynamical parameters

In [ ]:
# Chaotic RNN
params1 = {'N_neurons': 1500,
          'h': 0.1,
          'g': 3,
          'tau': 3,
          'dale': False
          }

# Adjusted chaotic RNN + Dale's law
params2 = {'N_neurons': 1500,
          'h': 0.1,
          'g': 3,
          'g_I': 1,
          'tau': 3,
          'x_s': 0.1,
          'dale': True
          }

# Kuramoto-Sakaguchi
params3 = {'N_neurons': 1500,
          'h': 0.1,
          'coupling': 2000,
          'tau': 100,
          'alpha': 50,
          'std': 0.1
          }

# Binary propagation
params4 = {'N_neurons': 1500,
          'h': 0.1,
          'P_activation': 0.15,
          'P_deactivation': 0.5
          }

# LIF
params5 = {'N_neurons': 1500,
         'h': 0.1,
         'g_L': 0.15,
         'E_L': -75,
         'E_E': -40,
         'E_I': -90,
         'V_thresh':-55,
         'V_reset':-75,
         'tau': 0.1,
         'I_E': 5,
         'I_I': 0,
         'sigma_E': 0,
         'sigma_I': 0,
         'delta_t': 0.025
         }

dynamics_list = [ChaoticRNN, AdjustedChaoticRNN, KuramotoSakaguchi, BinaryNetwork, LIFNetwork]
params_list = [params1, params2, params3, params4, params5]
calcium = [False, False, False, False, True]

In [ ]:
N_replicates = 50
N_averages = 25
T = 1000
N_modes = 20

mapping_matrices = [[], [], [], [], []]
r_geometric_scores = [[], [], [], [], []]
corrdist_curves_avg = [[], [], [], [], []]
corrdist_curves_std = [[], [], [], [], []]
corr_dist_correlation = [[], [], [], [], []]

for _ in tqdm(range(N_replicates)):
    for i in range(len(dynamics_list)):
    
        # Initializing simulator
        simulator = Simulator(ellipse,
                              dynamics_list[i],
                              params_list[i])
    
        # Running multiple simulations to generate average correlation matrix
        C = simulator.compute_average_correlations(n_iters=N_averages,
                                                   T=T,
                                                   calcium=calcium[i],
                                                   tau_calcium=10,
                                                   verbose=False)
    
        # Computing stuff from average correlation matrix C
        mode_similarity, _ = simulator.compute_geometric_mapping(C, N_modes=N_modes)
        r_geometric = np.mean(np.diag(np.abs(mode_similarity)))
        curve_avg, curve_std = simulator.compute_correlation_curve(C, d_max=0.5)

        d = compute_distances(simulator.coordinates, simulator.coordinates)
        triangle = np.triu_indices(d.shape[0], 1)
        

        mapping_matrices[i].append(mode_similarity)
        r_geometric_scores[i].append(r_geometric)
        corrdist_curves_avg[i].append(curve_avg)
        corrdist_curves_std[i].append(curve_std)
        corr_dist_correlation[i].append(pearsonr(C[triangle], d[triangle])[0])

In [ ]:
np.save('../Results/supp_dynamics_matrices.npy', mapping_matrices)
np.save('../Results/supp_dynamics_r_geometric.npy', r_geometric_scores)
np.save('../Results/supp_dynamics_corrdist_avg.npy', corrdist_curves_avg)
np.save('../Results/supp_dynamics_corrdist_std.npy', corrdist_curves_std)
np.save('../Results/supp_dynamics_corrdist_relationship.npy', corr_dist_correlation)

# Part 3: Rendering figure

Now we load the results and render the supplementary figure.

In [ ]:
from visualization import *
import matplotlib.ticker as ticker

In [ ]:
timeseries, correlations, mode_similarity = [], [], []

for i in range(5):
    timeseries.append(np.load(f'../Results/supp_dynamics_timeseries_dynamics{i+1}.npy'))
    correlations.append(np.load(f'../Results/supp_dynamics_correlations_dynamics{i+1}.npy'))
    mode_similarity.append(np.load(f'../Results/supp_dynamics_modesimilarity_dynamics{i+1}.npy'))

timeseries[2] = timeseries[2][:, 100:] # Removing dynamics onset
timeseries[4] = timeseries[4][:, 100:] # Removing dynamics onset

mapping_matrices = np.load('../Results/supp_dynamics_matrices.npy')
r_geometric_scores = np.load('../Results/supp_dynamics_r_geometric.npy')
corrdist_curves_avg = np.load('../Results/supp_dynamics_corrdist_avg.npy')
corrdist_curves_std = np.load('../Results/supp_dynamics_corrdist_std.npy')
corr_dist_correlation = np.load('../Results/supp_dynamics_corrdist_relationship.npy')

In [ ]:
den = 300

colors = [[1, 0, 0],
         [178/den, 161/den, 255/den],
         [44/den, 166/den, 255/den],
         [47/den, 216/den, 180/den],
         [255/den, 183/den, 0/den]]

ylim = [[0.1, 0.30],
       [0.05, 0.50],
       [0.07, 0.80],
       [0.03, 0.15],
       [0.03, 0.4]]

In [ ]:
cmaps = []
for c in colors:
    cmaps.append(make_cmap([[1, 1, 1], c], [0, 1]))

In [ ]:
fig = PaperFigure(figsize=(7, 9), dpi=600)

fig.set_tick_length(1)
fig.set_font_size(6)
fig.add_background()

# Adding panels -------------------------------

w = 1
ratio = 1.75
pad = (7 - (ratio + 3) * w) / 3
ypad = 0.35

for i in range(5):
    fig.add_axes(f'timeseries{i}', (0, i * (w + ypad)), ratio * w, w)
    fig.add_axes(f'correlations{i}', (ratio * w + pad, i * (w + ypad)), w, w)
    fig.add_axes(f'mapping{i}', ((1 + ratio) * w + 2 * pad, i * (w + ypad)), w, w)
    fig.add_axes(f'corrdist{i}', (6.2, i * (w + ypad)), w, w)

fig.add_axes('barchart', (0, 5 * (w + ypad)), 2.25, 1.5 * w)
fig.add_axes('scatter', (4, 5 * (w + ypad)), 2.25 * w, 1.5 * w)


fig.set_line_thickness(0.6)

# Content -------------------------------------

for i in range(5):
    
    ax = fig.axes[f'timeseries{i}']
    vmin = np.percentile(timeseries[i], 10)
    vmax = np.percentile(timeseries[i], 95)
    ax.imshow(timeseries[i], cmap='binary', aspect='auto', vmin=vmin, vmax=vmax, interpolation='None')
    ax.set_xticks([])
    ax.set_yticks([])

    ax = fig.axes[f'correlations{i}']
    vmin = np.percentile(correlations[i], 10)
    vmax = np.percentile(correlations[i], 95)
    ax.imshow(correlations[i], cmap='Reds', vmin=vmin, vmax=vmax)
    ax.set_xticks([])
    ax.set_yticks([])

    ax = fig.axes[f'mapping{i}']
    ax.imshow(np.mean(np.abs(mapping_matrices[i]), axis=0), cmap='Reds', vmin=0, vmax=1)
    #ax.imshow(np.abs(mode_similarity[i]), cmap='Reds', vmin=0, vmax=1)
    ax.set_xticks([])
    ax.set_yticks([])

    ax = fig.axes[f'corrdist{i}']
    bins = np.linspace(0, 0.5, 21, endpoint=True)
    d = bins[:-1] + (np.diff(bins) / 2)
    d[0] = 0
    curve_avg = np.mean(corrdist_curves_avg[i], axis=0)
    curve_std = np.mean(corrdist_curves_std[i], axis=0)
    ax.plot(d, curve_avg, color='black', linewidth=1)
    #ax.fill_between(d, curve_avg - curve_std, curve_avg + curve_std, color='black', alpha=0.2, edgecolor=None)
    ax.spines[['top', 'right']].set_visible(False)
    ax.axvline(0.1, color='black', linewidth=0.5, linestyle='--')
    ax.set_xlim([0, 0.4])
    ax.set_ylim([ylim[i][0], ylim[i][1]])
    ax.set_yticks([ylim[i][0], ylim[i][1]])

ax = fig.axes['barchart']
for i in range(5):
    ax.scatter(np.random.uniform(i-0.1, i+0.1, len(r_geometric_scores[i])), r_geometric_scores[i], color=colors[i], edgecolor='None', s=5, alpha=0.5)
    bp = ax.boxplot(r_geometric_scores[i], positions=[i + 0.4], patch_artist=True, widths=0.1, whis=[5, 95])
    set_boxplot_color(bp, colors[i], marker='None', linewidth=0.75)
ax.set_xlim([-0.5, 5])
ax.set_ylim([0.35, 0.775])
ax.set_xticks([])
ax.spines[['top', 'right']].set_visible(False)

ax = fig.axes['scatter']
colors_scatter = [colors[0]] * 50 + [colors[1]] * 50 + [colors[2]] * 50 + [colors[3]] * 50 + [colors[4]] * 50
x = np.concatenate(np.abs(corr_dist_correlation))
y = np.concatenate(r_geometric_scores)
a, b = np.polyfit(x, y, 1)
ax.scatter(np.concatenate(np.abs(corr_dist_correlation)), np.concatenate(r_geometric_scores), color=colors_scatter, alpha=0.5, s=5, edgecolor='None')
x = np.linspace(np.min(np.abs(corr_dist_correlation)), np.max(np.abs(corr_dist_correlation)), 30)
ax.plot(x, a * x + b, color='black', linewidth=1)
ax.spines[['top', 'right']].set_visible(False)
#ax.set_xlim([0.125, 0.28])
#ax.set_ylim([0.35, 0.775])

# Displaying ----------------------------------

fig.show()

In [ ]:
fig.save('../Figures/supp_dynamics_incomplete.svg')

#### Manually written values

Correlation values

In [ ]:
for c in correlations:
    vmin = np.percentile(c[i], 10)
    vmax = np.percentile(c[i], 95)
    print(vmin, vmax)

Statistical test (One-way ANOVA)

In [ ]:
groups_ANOVA_Tukey(r_geometric_scores)

Linear regression

In [ ]:
x = np.concatenate(np.abs(corr_dist_correlation))
y = np.concatenate(r_geometric_scores)
pearsonr(x, y)